In [1]:
from src.constants import p, a, b
from src.CurvePoint import CurvePoint
from src.EllipticCurve import EllipticCurve


from src.Emitter import Emitter
from src.Receiver import Receiver
from src.Interceptor import Interceptor


from src.utils import initPublic

In [2]:
P = CurvePoint(a, b, 8, 1)
public = initPublic(P)

In [3]:
A = Emitter("Ceci est un message secret", public)
B = Receiver(public)

In [4]:
A.encodeMsg(public)

In [5]:
B.decodeMsg(public)

'Ceci est un message secret '

In [6]:
I = Interceptor()
Q = 112*P
print(Q)


Point de E(Z/317Z) : (78 [317], 15 [317])


In [7]:
I.log_discret(P, Q)

112